# IMPORTS

In [7]:
from sklearn.ensemble import RandomForestClassifier
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_score, recall_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import MultinomialNB
# from roc import plot_roc
from sklearn.metrics import roc_curve

import my_pickle as mp

# READ IN DATA

In [14]:
# read in user data
user_df = mp.unpickle_it("data_user")
convo_df = mp.unpickle_it("data_convo")
len(user_df)

35343

In [ ]:
# # read in message data
# path_response = "/Users/gandalf/Documents/data/data_response.pkl"
# response_df = pd.read_pickle(path_response)
# len(response_df)

In [13]:
# # read in message data
# path_response = "/Users/gandalf/Documents/data/data_response.pkl"
# response_df = pd.read_pickle(path_response)
# response_df.head(5)

In [15]:
# response rate?
convo_df.response.sum()/len(convo_df)

0.1081958581958582

In [6]:
# # get recipients
# outside_NA_conv['first_ten'] = outside_NA_conv.conv_id.apply(lambda x: x[:10])
# outside_NA_conv['last_ten'] = outside_NA_conv.conv_id.apply(lambda x: x[10:])

# for index, row in outside_NA_conv.iterrows():
#     if row.second_uid == None:
#         if row.first_ten == row.first_uid: row.second_uid = row.last_ten
#         elif row.last_ten == row.first_uid: row.second_uid = row.first_ten
#         else: print('uh')

NameError: name 'outside_NA_conv' is not defined

In [16]:
convo_df[convo_df.response].head()

,conv_id,response,first_uid,second_uid,first_mid,second_mid,timestamp
16296,rKdn0zblFiRVDUEmDHyQ,True,rKdn0zblFi,RVDUEmDHyQ,-KWKghvlpF67UOv-iXXO,-KWKg_ybWV4qbG02NYrb,1478906535730
1069,8Ws3QWmC4B52lmw7KQcq,True,8Ws3QWmC4B,52lmw7KQcq,-KWPb8hd_DbLpY1eIUlT,-KWQNe2ghZCGV0aywnKp,1478988962771
14397,nQYG2Z9elg8Ws3QWmC4B,True,nQYG2Z9elg,8Ws3QWmC4B,-KWZ6j5vHnNWPD90JJ18,-KWUqwqvF7RfcRaEqtL6,1479148499453
14165,n0FApJHyVPDlDAwxmm76,True,n0FApJHyVP,DlDAwxmm76,-KWZNYj1myibonhSkohK,-KWZO8p49vmFvYOCohJt,1479152909431
14398,nQYG2Z9elgIR6imqlZRG,True,IR6imqlZRG,nQYG2Z9elg,-KWZt1Pv-6EBI1-Uz-Aj,-KWZvjJmZ9kI-nAAInho,1479161423843


In [17]:
# make it simple
simple_cols = ['first_uid','second_uid','response']
simple_df = convo_df[simple_cols]
# tf = {True:1, False:0, 1:1, 0:0}
# simple_df.response = simple_df.response.map(tf)
simple_df.head()

,first_uid,second_uid,response
16296,rKdn0zblFi,RVDUEmDHyQ,True
9241,aXnWmLjpaE,RVDUEmDHyQ,False
1069,8Ws3QWmC4B,52lmw7KQcq,True
9414,b1HYtkC11e,92RR2gpmjr,False
4142,KnmYDsn5Q4,92RR2gpmjr,False


In [18]:
# build up wiht user info
ad,sg,sr,sc,sn,ss = [],[],[],[],[],[]

for index, row in simple_df.iterrows():
    first = user_df.loc[str(row.first_uid)]
    second = user_df.loc[str(row.second_uid)]
    ad.append(abs(first.age - second.age))
    sg.append(first.gender == second.gender)
    sr.append(first.inRelationship == second.inRelationship)
    sc.append(first.isClean == second.isClean)
    sn.append(first.isNight == second.isNight)
    ss.append(first.isStudent == second.isStudent)

simple_df['age_dif'] = ad
simple_df['same_gender'] = sg
simple_df['same_relate'] = sr
simple_df['same_clean'] = sc
simple_df['same_night'] = sn
simple_df['same_student'] = ss

# filled na at some point, should undo

simple_df.head(2)

/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
/Users/gandalf/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the docum

,first_uid,second_uid,response,age_dif,same_gender,same_relate,same_clean,same_night,same_student
16296,rKdn0zblFi,RVDUEmDHyQ,True,11.0,True,False,False,True,False
9241,aXnWmLjpaE,RVDUEmDHyQ,False,11.0,False,True,False,True,False


In [19]:
# change into x and y
X = simple_df
y = X.response.map({True:1,False:0})
X = X.drop(['first_uid','second_uid','response'], axis=1)
X.head()
y.sum()

1233

In [20]:
# train, test split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 17)

In [22]:
# Use sklearn's RandomForestClassifier to build a model of your data
model = RandomForestClassifier()
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


print("Prediction:   {}".format(model.predict(X_test)))
print("Actual:       {}".format(y_test.values))

print("\nModel recall: {} (Of things the model says are 1's, how many are 1's?)".format(recall_score(y_test, y_pred)))
print("Model precision: {} (Of things that are 1's, how many does the model says are 1's?)".format(precision_score(y_test, y_pred)))
print("Score of model: {}".format(model.score(X_test, y_test)))
print("Score to beat: {}".format(1-y_test.sum()/len(y_test)))

print ("\nconfusion matrix:")
print ("   N  P")
print (confusion_matrix(y_test, y_pred))

Prediction:   [0 0 0 ..., 0 0 0]
Actual:       [0 0 0 ..., 1 0 0]

Model recall: 0.00906344410876133 (Of things the model says are 1's, how many are 1's?)
Model precision: 0.2727272727272727 (Of things that are 1's, how many does the model says are 1's?)
Score of model: 0.8820638820638821
Score to beat: 0.8838188838188838

confusion matrix:
   N  P
[[2510    8]
 [ 328    3]]


In [23]:
# Build the RandomForestClassifier again setting the out of bag parameter to be true
model = RandomForestClassifier(n_estimators=30, oob_score=True)
model.fit(X_train, y_train)
print ("accuracy score:", model.score(X_test, y_test))
print ("out of bag score:", model.oob_score_)

accuracy score: 0.881361881362
out of bag score: 0.890955890956


In [ ]:
# Use sklearn's model to get the feature importances
feature_importances = np.argsort(model.feature_importances_)
print("top five:", list(simple_df.columns[feature_importances[-1:-6:-1]]))

In [ ]:
# Calculate the standard deviation for feature importances across all trees

n = 10 # top 10 features

#importances = forest_fit.feature_importances_[:n]
importances = model.feature_importances_[:n]
std = np.std([tree.feature_importances_ for tree in model.estimators_],
             axis=0)
indices = np.argsort(importances)[::-1]
features = list(X.columns[indices])

print("Feature ranking:")

for f in range(5):
    print("%d. %s (%f)" % (f + 1, features[f], importances[indices[f]]))


In [ ]:
features

In [ ]:
# Plot the feature importances of the forest
plt.figure()
plt.title("Feature importances")
plt.bar(range(6), importances[indices], yerr=std[indices], color="r", align="center")
plt.xticks(range(6), features, rotation=45)
plt.xlim([-1, 5])
plt.show()

In [ ]:
# Try modifying the number of trees

num_trees = range(5, 50, 5)
accuracies = []
for n in num_trees:
    tot = 0
    for i in range(5):
        rf = RandomForestClassifier(n_estimators=n)
        rf.fit(X_train, y_train)
        tot += rf.score(X_test, y_test)
    accuracies.append(tot / 5)
plt.plot(num_trees, accuracies)
plt.xlabel="num_trees"
plt.ylabel="accuracy"
plt.show()

In [ ]:
# Try modifying the number of trees
for nn in range(10):
    num_trees = range(5, 50, 5)
    accuracies = []
    for n in num_trees:
        tot = 0
        for i in range(5):
            rf = RandomForestClassifier(n_estimators=n)
            rf.fit(X_train, y_train)
            tot += rf.score(X_test, y_test)
        accuracies.append(tot / 5)
    plt.plot(num_trees, accuracies)
plt.xlabel="num_trees"
plt.ylabel="accuracy"
plt.show()

In [ ]:
# Modifying the max features parameter
for nn in range(10):
    num_features = range(2, len(X.columns))
    accuracies = []
    for n in num_features:
        tot = 0
        for i in range(5):
            rf = RandomForestClassifier(max_features=n)
            rf.fit(X_train, y_train)
            tot += rf.score(X_test, y_test)
        accuracies.append(tot / 5)
    plt.plot(num_features, accuracies)
    plt.xlabel="num_features"
    plt.ylabel="accuracy"
plt.show()

In [ ]:
# Run all the other classifiers that we have learned so far in class
def get_scores(classifier, X_train, X_test, y_train, y_test, **kwargs):
    model = classifier(**kwargs)
    model.fit(X_train, y_train)
    y_predict = model.predict(X_test)
    return model.score(X_test, y_test), \
           precision_score(y_test, y_predict), \
           recall_score(y_test, y_predict)

print ("    Model,                Accuracy, Precision, Recall")
print ("    Random Forest:       ", get_scores(RandomForestClassifier, X_train, X_test, y_train, y_test, n_estimators=25, max_features=5))
print ("    Logistic Regression: ", get_scores(LogisticRegression, X_train, X_test, y_train, y_test))
print ("    Decision Tree:       ", get_scores(DecisionTreeClassifier, X_train, X_test, y_train, y_test))
print ("    Naive Bayes:         ", get_scores(MultinomialNB, X_train, X_test, y_train, y_test))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
from scipy import interp
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler

In [ ]:
def plot_roc(X, y, clf_class, title, **kwargs):
# def plot_roc(X, y, clf_class, kwargs):
    scaler = StandardScaler()
    X = scaler.fit_transform(X)
    kf = KFold(len(y), n_folds=5, shuffle=True)
    y_prob = np.zeros((len(y),2))
    mean_tpr = 0.0
    mean_fpr = np.linspace(0, 1, 100)
    all_tpr = []
    for i, (train_index, test_index) in enumerate(kf):
        X_train, X_test = X[train_index], X[test_index]
        y_train = y[train_index]
        clf = clf_class(**kwargs)
        clf.fit(X_train,y_train)
        # Predict probabilities, not classes
        y_prob[test_index] = clf.predict_proba(X_test)
        fpr, tpr, thresholds = roc_curve(y[test_index], y_prob[test_index, 1])
        mean_tpr += interp(mean_fpr, fpr, tpr)
        mean_tpr[0] = 0.0
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=1, label='ROC fold %d (area = %0.2f)' % (i, roc_auc))
    mean_tpr /= len(kf)
    mean_tpr[-1] = 1.0
    mean_auc = auc(mean_fpr, mean_tpr)
    plt.plot(mean_fpr, mean_tpr, 'k--',label='Mean ROC (area = %0.2f)' % mean_auc, lw=2)

    plt.plot([0, 1], [0, 1], '--', color=(0.6, 0.6, 0.6), label='Random')
    plt.xlim([-0.05, 1.05])
    plt.ylim([-0.05, 1.05])
#     plt.xlabel('False Positive Rate')
#     plt.ylabel('True Positive Rate')
    plt.title(title + 'ROC')
    plt.legend(loc="lower right")
    plt.show()

In [ ]:
xfake, yfake = np.random.rand(100,6), np.random.randint(0,2, size=(100,))

In [ ]:
plt.figure(figsize=(20,16))
print ("Visualize the roc curve of each model")
plot_roc(xfake, yfake, RandomForestClassifier, 'Random_Forest', n_estimators=25, max_features=5)
#plot_roc(X, y, LogisticRegression, 'Logistic_Regrssion')
#plot_roc(X, y, DecisionTreeClassifier, 'Decision_Tree')
#plot_roc(X, y, MultinomialNB, 'Naive_Bayes') error
print('\nPlotting completed.')


In [ ]:
print ("Visualize the roc curve of each model")
plot_roc(X, y, RandomForestClassifier, 'Random_Forest', n_estimators=25, max_features=5)
plot_roc(X, y, LogisticRegression, 'Logistic_Regrssion')
plot_roc(X, y, DecisionTreeClassifier, 'Decision_Tree')
#plot_roc(X, y, MultinomialNB, 'Naive_Bayes') error
print('\nPlotting completed.')

In [ ]:
to do:
    roc curve
    split data
    how to t/t
    